# Class
---

- We declare a class to abstract over objects. A class is a "template" to create objects, and objects of a class share the same type (same name as class) and have common properties. In the note `01-basics`, we talked about how to declare Singleton objects. It's a useful construct for certain usecases but has this limitation that you may need to define separate object even for the same properties, and even then we won't be able to write a general method to accept any object with those same properties.

Let's see an example,

In [1]:
object Person1 {
  val name = "John"
  def say(greet: String) = name + " greets " + greet
}

def personGreets(msg: String, p: Person1.type): Unit = println(p.say(msg))

personGreets("hi there", Person1)

John greets hi there


defined object Person1
defined function personGreets

In [1]:
object Person2 {
  val name = "John"
  def say(greet: String) = name + " greets " + greet
}

personGreets("hi there", Person2)

cmd1.sc:6: type mismatch;
 found   : ammonite.$sess.cmd1.Helper.Person2.type
 required: ammonite.$sess.cmd0.instance.Person1.type
val res1_1 = personGreets("hi there", Person2)
                                      ^Compilation Failed

: 

We can notice two issues in above example,

1) we had to write down two objects even though they share same "structure" (no code re-usability)
2) `personGreets` method couldn't accept `Person2` object even though it has all required properties (different type)

We can solve this problem with `class` construct. As mentioned, class provides general structure for objects (issue #1 resolved), and all objects of the class have the same type (issue #2 resolved)

In [2]:
class Person {
    val name = "John"
    def say(greet: String) = name + " greets " + greet
}

defined class Person

Notice that it has same syntax as for declraing an `object`, but unlike object, we can't use class name in an expression and like an object declaration, a class declaration binds a name and it's not an expression.

- we can construct new `instances` of a class using `new` operator.

Let's create two new objects and create general method to take any instance of type `Person` to print the greeting message.

In [3]:
val john1: Person = new Person // created object from class
val john2: Person = new Person // created another object from class

def personGreets(msg: String, p: Person): Unit = println(p.say(msg))

personGreets("hi there", john1)
personGreets("hi there", john2)

John greets hi there
John greets hi there


john1: Person = ammonite.$sess.cmd1$Helper$Person@49cf12ab
john2: Person = ammonite.$sess.cmd1$Helper$Person@7b0db6c9
defined function personGreets

But there is still an issue. We don't have an ability to create an object of `Person` type with any name we like. Right now, the name "John" is fixed and we want an ability to pass a name of a person while creating new object, a `constructor`. Similar to method parameter, class allows us to pass parameter, and we can use them to instantiate object. This params are called `constructor parameters`. 

In [4]:
class Person(nm: String) {
    val name = nm
    def say(greet: String) = name + " greets " + greet
}

val john = new Person("John")
val jil = new Person("Jil")

def personGreets(msg: String, p: Person): Unit = println(p.say(msg))

personGreets("hi there", john)
personGreets("hi there", jil)

John greets hi there
Jil greets hi there


defined class Person
john: Person = ammonite.$sess.cmd3$Helper$Person@1685584c
jil: Person = ammonite.$sess.cmd3$Helper$Person@11d80fb2
defined function personGreets

- Syntax: 
    - declaration: `class <name>(<param>:<type>, ...) { <body> }`
    - create object: `new <class name>(<value>, ...)`
    
- Notes:
    - constructor params are not necessarly class members, but using `var`, `val` or `def` we can make then class memebers
    - we can use `parameter names` to pass param values in arbitrary order
    - we can have default values to a parameter, and if we omit a param while constructing an object, default value will be used to create an instance    
    
#### type hierarchy
---

- Scala's type system is unified which means value (primitive) types and reference types have the common super type. The diagram below illustrates a subset of the type hierarchy.
<img src="images/unified-types-diagram.svg" style="display: block; margin-left: auto; margin-right: auto; width: 80%;" />
<sup>image credit: Scala Lang Docs</sup>

- `Any` is the super type of all types, and defines common methods that all types must have.
- `AnyVal` and `AnyRef` are direct sub type of `Any`, and they are super type of value types and reference types, respectively.
- value types
    - JVM's primitive types are subtype of `AnyVal`, and becuase Scala is pure object-oriented language, even primitives values are being treated as objects.
    - example, `Int`, `Long`, `Double` etc..
- reference types
    - all non-value types are defined as reference types, and every user defined type in Scala is a subtype of `AnyRef`<sup>1</sup>.
- `Null` is subtype of every reference types. (null value)
- `Nothing` is subtype of all types.
    - there is no value that has type `Nothing`.
    - common use case is to signal non-termination such as expcetion thrown or infinite loop.
    ```scala
    scala> :type throw new Exception("some error")
    Nothing
    ```

#### conditional expression and its type inference
---

- the conditional `if` statement returns a value because it's an expression. Because it can hold two expressions of different types, it can help us understand type hierarchy.
- conditional expression is made of three expressions, a condition, true branch and false branch.
    - Syntax: `if (<condition>) { <trueBranch> } else { <falseBranch> }`
    - curly braces are optional if branches are single expression
- the type of conditional expression is `least common type` of types of both branches.

```scala
scala> :type if (false) 1 else ()
AnyVal
scala> :type if (false) "hello" else jil
AnyRef
scala> :type if (false) 1 else "hello"
Any
```

- if `trueBranch` is reference type and `falseBranch` is `null`, then the type `trueBranch` will be the type of whole expression.
- it is also possible to write if expression without `else` part
    - Syntax: `if (<condition>) { <trueBranch> }`
    - it will be similar expression as `if (<condition>) { <trueBranch> } else { () }`
    
---

[1] User can define value classes, those types will be subtype of `AnyVal` (rather than `AnyRef`).

---

### Exercise

1) Why does `object` construct not enough to write business logic, and what construct Scala does provide to overcome those limitations?

2) What operator do we use to create a new object of a class? (choose one)
    - `create`
    - `make`
    - `new`
    - none of the above

3) What is a constructor, and what's the difference between constructor parameters and class members?

4) What will be the type of `if (true) throw new Exception("some error") else null` expression? (choose one)
    - `Nothing`
    - `Null`
    - `AnyRef`
    - `Any`

5) Define a `Counter` class. It should take the `name` of the counter and initial `count` value on constructing an object. It should be able to increment and decrement the count, and should query "current count.